#### Data Network Category (Download, Upload, LDRs)

```{r Data Network Category, echo=FALSE, fig.align="left", fig.asp=0.62, fig.height=5, fig.width=10, message=FALSE, warning=FALSE, ng=FALSE}
network_category_curr <- dbGetQuery (rsr_db, sqlInterpolate(rsr_db,
                                                            " with best_network_type as (
  select pro.product_period, c.friendly_name as carrier,
  md2.fn_get_best_network_type(ts.test_type_id, ts.net_types, tea.network_types, tea.call_network_type, tea.nr_status_filtered, tea.nr_bearer_status_filtered,tea.nr_bearer_allocation_status_filtered,',') best_network_type
  from prod_rsr_partitions.test_event_aggr_?curr_csid tea
  join prod_ms_partitions.test_summary_?curr_csid ts using (test_event_id)
  join md2.carriers c on (c.carrier_id = ts.carrier_id)
  join md2.product_periods pro using(product_period_id)

  <!-- where c.friendly_name NOT IN ('Dish') AND  ts.blacklisted = false and ts.flag_valid = true and ts.collection_set_period_id is not null and ts.test_type_id in (20,19,26)
), -->

data_network_category as (
select product_period, carrier,
    case when best_network_type in ('NR SA','NR NSA') then '5G'
    when best_network_type in ('NR SA, LTE','NR NSA, LTE') then 'Mixed-5G'
      when best_network_type in ('LTE') then 'LTE'
    else 'Non-LTE' end as network
from best_network_type
)
select product_period, carrier, network, count(*) as count, round(100 * count(*) / sum(count(*)) over (partition by carrier),1) as percent
from data_network_category
group by product_period, carrier, network
order by carrier, case when network = '5G' then 1 when network = 'Mixed-5G' then 2 when network = 'LTE' then 3 when network = 'Non-LTE' then 4 end", curr_csid = curr_csid))

#head(network_category_curr)

network_category_comp <- dbGetQuery (rsr_db, sqlInterpolate(rsr_db,
                                                            " with best_network_type as (
  select pro.product_period, c.friendly_name as carrier,
  md2.fn_get_best_network_type(ts.test_type_id, ts.net_types, tea.network_types, tea.call_network_type, tea.nr_status_filtered, tea.nr_bearer_status_filtered,tea.nr_bearer_allocation_status_filtered,',') best_network_type
  from prod_rsr_partitions.test_event_aggr_?comp_csid tea
  join prod_ms_partitions.test_summary_?comp_csid ts using (test_event_id)
  join md2.carriers c on (c.carrier_id = ts.carrier_id)
  join md2.product_periods pro using(product_period_id)

  <!-- where c.friendly_name NOT IN ('Dish') AND  ts.blacklisted = false and ts.flag_valid = true and ts.collection_set_period_id is not null and ts.test_type_id in (20,19,26)
), -->

data_network_category as (
select product_period, carrier,
    case when best_network_type in ('NR SA','NR NSA') then '5G'
    when best_network_type in ('NR SA, LTE','NR NSA, LTE') then 'Mixed-5G'
      when best_network_type in ('LTE') then 'LTE'
    else 'Non-LTE' end as network
from best_network_type
)
select product_period, carrier, network, count(*) as count, round(100 * count(*) / sum(count(*)) over (partition by carrier),1) as percent
from data_network_category
group by product_period, carrier, network
order by carrier, case when network = '5G' then 1 when network = 'Mixed-5G' then 2 when network = 'LTE' then 3 when network = 'Non-LTE' then 4 end", comp_csid = comp_csid))

#head(network_category_comp)

network_category <- rbind(network_category_curr, network_category_comp)
network_category<- reshape(network_category[,-c(4)], idvar = c("carrier","product_period"), timevar = "network", direction = "wide")
col_order <- c("product_period", "carrier", "percent.5G","percent.Mixed-5G", "percent.LTE","percent.Non-LTE")
col_order <- col_order[col_order %in% names(network_category)]
network_category<- network_category[,col_order]
network_category[is.na(network_category)] <-0
if(length(names(network_category))==6){
  names(network_category) <- c("period", "carrier", "5G (%)", "Mixed-5G (%)", "LTE (%)", "Non-LTE (%)")
} else {
  names(network_category) <- c("period", "carrier", "5G (%)", "Mixed-5G (%)", "LTE (%)")
}
network_category <- arrange(network_category, carrier)
kable(network_category, row.names = FALSE) %>% kable_styling(position = "center",bootstrap_options = "bordered", latex_options = "striped", font_size = 16, full_width = FALSE)
```

#head(network_category)

<!-- select section below and hit ctrl+shift+c to comment out for US markets and comment in for UK markets -->
  